In [68]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

max_size = 1000

# get the dataset
data = open( "blackjack2-2.data").readlines()
tags = open( "blackjack2-2.tags").readlines()
data_clean = []
tags_clean = []
#strip whitespace
first = True
i = 0
for datum in data:
	i += 1
	if i >= max_size:
		break

	# skip empty line first
	if first:
		first = False
		continue
	clean_datum = datum[:datum.index('\n')].strip()
	clean_datum = clean_datum[1:-1].split(',')
	clean_datum[0] = int( clean_datum[0] )
	clean_datum[1] = int( clean_datum[1][1:] )
	# print( clean_datum )
	data_clean = data_clean + [ clean_datum ]

first = True
i=0
for tag in tags:
	i += 1
	if i >= max_size:
		break
	if first:
		first = False
		continue
	tag = tag[:tag.index('\n')]

	if tag == "h":
		tags_clean = tags_clean + [ 1.0 ]
	else:
		tags_clean = tags_clean + [ 0.0 ]

scaler = MinMaxScaler()
data_clean = scaler.fit_transform(data_clean)


size = int( (0.80) * max_size )

train_data = np.array( data_clean[1:size] )
train_tags = np.array( tags_clean[1:size] )
test_data = np.array( data_clean[size:] )
test_tags = np.array( tags_clean[size:] )

In [69]:
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_machine_learning.algorithms.classifiers import VQC




feature_map = ZZFeatureMap(feature_dimension=2, reps=1)
ansatz = RealAmplitudes(num_qubits=2, reps=3)
optimizer = COBYLA(maxiter=100)
sampler = Sampler()

vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer
)

/var/folders/q9/qd1csgkx47vdf02qrmzh6nbm0000gn/T/ipykernel_68374/2799304377.py:12: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [70]:
print("Training the VQC...")
vqc.fit(train_data, train_tags)
print("Training complete.")

Training the VQC...
Training complete.


In [71]:
score = vqc.score(test_data, test_tags)
print(f"Model Accuracy on Test Data: {score * 100:.2f}%")

Model Accuracy on Test Data: 61.62%


In [72]:
player_hand = 16
dealer_card = 10

new_data_point = scaler.transform([[player_hand, dealer_card]])
prediction = vqc.predict(new_data_point)
decision = "Hit" if prediction == 1 else "Stay"

print(f"Player: {player_hand}, Dealer: {dealer_card}")
print(f"Quantum Decision: {decision}")

Player: 16, Dealer: 10
Quantum Decision: Stay
